# Test Cases



In [1]:
import os
import sys
from mylist import *
from mycrm import *
from myserver import *
from myvm import *
from inspect import currentframe, getframeinfo

# test.py for toy cloud assignment.
# feel free to comment the relevant parts in main function.

## test helper functions

global test_no
global crm
test_no=0
def mycheck(result, msg, lno=0):
    global test_no
    test_no += 1
    cf=currentframe()
    l1=cf.f_back.f_lineno
    if lno:
        l=f'{lno},{l1}'
    else:
        l=f'{l1}'
    if result:
        print(f'{l} tno: {test_no} {msg} PASSED ')
    else:
        print(f'{l}: tno: {test_no} {msg} FAILED ')
        sys.exit(1)

# template crmstat to copy and change values.
#mys=crm_stat(stotal=1,sidle=1,sactive=0,sfail=0,vtotal=1,vprovision=1,vactive=0,'vidle', vdeleted=0)
#print(mys)


def check_stat(s1):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_stat()
    mycheck((s==s1),f'{s1} == {s}', lno)

def check_server_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers(state)
    mycheck(len(s) == count, f'server_count {len(s)} == {count} for state {state}', lno)

    s=crm.get_servers('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'server_count {c} == {count} for state {state}', lno)

def check_server_state(sname, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    assert(state != 'any')
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)
    s=crm.get_servers(state)
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)

def check_vm_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_vms(state)
    mycheck(len(s) == count, f'vm count {len(s)} == {count} for state {state}', lno)

    s=crm.get_vms('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms(state)
        for j in v:
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if state != 'any' and j.state != state:
                continue
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)



def check_server_vm_count(sr, count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=fs.get_vms('any')
    c=0
    for i in v:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 
    if state == 'any':
        return
    v=fs.get_vms(state)
    c=0
    for i in v:
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 

def check_server_vm_state(sr, vm, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    if f != 1:
        mycheck(0, f'vm {vm} in {sr} lookup', lno)
    mycheck(fv.state==state, f'vm: {fv.vc.name} state {fv.state} == {state}', lno) 

def check_novm(vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_vms('any')
    c=0
    for i in s:
        if i.vc.name == vm:
            c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if j.vc.name == vm:
                c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)

def check_server_novm(sr, vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    mycheck(f==0, f'vm: {vm}  in server {sr} not present', lno) 

aggr=namedtuple('aggr', 'ncpu mem')
vmt=namedtuple('vmt', 'ncpu mem')
testvm=namedtuple('testvm', 'vc pvm')

def get_available():
    available_cpus=0
    available_mem=0
    avails=[]
    s=crm.get_servers('any')
    for i in s:
        available_cpus += i.acpu
        available_mem += i.amem
        avails.append(sconf(i.sc.name, i.acpu, i.amem))

    print(avails)
    y=aggr(available_cpus, available_mem)
    return y
         

# 

# Last Test

In [21]:
global crm
crm=mycrm()
# 10 servers, 1000 vms
ps=[]
nservers=10
for i in range(0,nservers):
    sname=f's{i}'
    a = crm.add(sname, 150+i*50, 1000+2*i*200)
    ps.append(a)
print (f'Physical Servers: {ps}')
total_memory=0
total_cpu=0
for i in ps:
    total_memory +=i.sc.mem
    total_cpu += i.sc.ncpu
total = aggr(total_cpu,total_memory)
print (total)
t1=vmt(2, 4)
t2=vmt(4,16)
t3=vmt(8,64)
t4=vmt(16,128)
t5=vmt(32,256)
t6=vmt(64,512)
t7=vmt(128,1024)
vmtypes=[t1, t2, t3, t4, t5, t6, t7]
print(vmtypes)
# keep allocating until we run out of space
i=0
v=0
vms=[]
pvms=[]
mod=len(vmtypes) 
while True:
    vname=f'vm{v}'
    ncpu=vmtypes[i].ncpu
    mem=vmtypes[i].mem
    vm = crm.provision_vm(vname, ncpu, mem)
    if vm == None:
        print(f'provisioning failed for {vmtypes[i]}')
        mod -= 1
        if mod == 0:
            break
    else:
        tvm=testvm(vm.vc,vm)
        vms.append(tvm)
        pvms.append(vm)
        v += 1
    i=(i+1)%mod


allocated_cpus=0
allocated_mem=0
#    print(vms)
for v in pvms:
    allocated_cpus += v.vc.ncpu 
    allocated_mem += v.vc.mem 

allocated=aggr(allocated_cpus, allocated_mem)
available=get_available()
print(f'allocated:{allocated} Total: {total} available:{available}')
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,vtotal=len(vms),vprovision=len(vms),vactive=0, vdeleted=0))

vmtypes=[t1, t2, t3, t4, t5, t6, t7]
for s in ps:
    check_server_state(s.sc.name, 'idle')
    check_vm_count(len(vms), 'provision')
    check_vm_count(len(vms), 'any')
    for v in s.get_vms('any'):
        check_server_vm_state(s.sc.name, v.vc.name, 'provision')
    check_server_vm_count(s.sc.name, s.nvms, 'provision')
    mycheck(((s.acpu < vmtypes[0].ncpu) or (s.amem < vmtypes[0].mem)),
            f'Server {s.sc.name, s.acpu, s.amem} available < min vm {vmtypes[0].ncpu}')
crm.provision_undo(10)
check_stat(crm_stat(stotal=nservers,sidle=nservers,sactive=0,sfail=0,vtotal=len(vms)-10,vprovision=len(vms)-10,vactive=0, vdeleted=10))

Physical Servers: [<myserver.myserver object at 0x7fab66825e80>, <myserver.myserver object at 0x7fab6694d2b0>, <myserver.myserver object at 0x7fab6694d6d0>, <myserver.myserver object at 0x7fab6694d820>, <myserver.myserver object at 0x7fab6694df70>, <myserver.myserver object at 0x7fab6694d940>, <myserver.myserver object at 0x7fab669306d0>, <myserver.myserver object at 0x7fab66930760>, <myserver.myserver object at 0x7fab66930e20>, <myserver.myserver object at 0x7fab66930700>]
aggr(ncpu=3750, mem=28000)
[vmt(ncpu=2, mem=4), vmt(ncpu=4, mem=16), vmt(ncpu=8, mem=64), vmt(ncpu=16, mem=128), vmt(ncpu=32, mem=256), vmt(ncpu=64, mem=512), vmt(ncpu=128, mem=1024)]
provisioning failed for vmt(ncpu=128, mem=1024)
provisioning failed for vmt(ncpu=64, mem=512)
provisioning failed for vmt(ncpu=32, mem=256)
provisioning failed for vmt(ncpu=8, mem=64)
provisioning failed for vmt(ncpu=8, mem=64)
provisioning failed for vmt(ncpu=4, mem=16)
provisioning failed for vmt(ncpu=2, mem=4)
[sconf(name='s9', ncpu

In [8]:
# verify that the last 10 vms are not present

for i in range(0,10):
    v=vms[-i-1]
    pv=pvms[-i-1]
#     check_novm(v.vc.name)
    print(v.vc.name)
    #mycheck((pv.state=='deleted'), f'vm {pv.vc.name} state {pv.state} == deleted')

vm119
vm118
vm117
vm116
vm115
vm114
vm113
vm112
vm111
vm110


In [19]:
for i in range(1):
    print('Yesy')

Yesy


In [22]:
for ser in crm.server_list:
    for vir in ser.vl:
        if vir.name=='vm119':
            ser.vl.delete(vir)

In [23]:
for ser in crm.server_list:
    for vir in ser.vl:
        if vir.name=='vm119':
            print(vir.name,ser.name)

vm119 s0


In [24]:
crm.server.new_vm_list

AttributeError: 'myserver' object has no attribute 'new_vm_list'

In [9]:
vm='vm119'
cf=currentframe()
lno=cf.f_back.f_lineno

s=crm.get_vms('any')
c=0
for i in s:
    if i.vc.name == vm:
        c += 1
mycheck(c == 0, f'vm {vm} not present', lno)

3461,10: tno: 255 vm vm119 not present FAILED 


SystemExit: 1

/home/blink/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:

c=0
s=crm.get_servers('any')
for i in s:
    v=i.get_vms('any')
    for j in v:
        if j.vc.name == vm:
            c += 1
mycheck(c == 0, f'vm {vm} not present', lno)

## Crm test

In [25]:
global crm
crm=mycrm()
a = crm.add("s1", 500, 1000)
print(a.sc)
check_server_count(1, 'any')
s=crm.get_servers('any')
mycheck((len(s) == 1), "get servers should return one entry")
check_server_count(1, 'idle')
check_server_state('s1', 'idle')
s=crm.get_stat()
check_stat(crm_stat(stotal=1,sidle=1))
crm.fail(a.sc.name)
check_stat(crm_stat(stotal=1,sfail=1))
check_server_state('s1', 'fail')
crm.unfail(a.sc.name)
check_stat(crm_stat(stotal=1,sidle=1))
check_server_state('s1', 'idle')
crm.delete_server(a.sc.name)
check_stat(crm_stat(sdeleted=1))
check_server_count(0, 'any')

print(f'*********crm_1server tests done *********')
print(f'*********crm_1server tests done *********')

sconf(name='s1', ncpu=500, mem=1000)
5,48 tno: 253 server_count 1 == 1 for state any PASSED 
5,56 tno: 254 server_count 1 == 1 for state any PASSED 
7 tno: 255 get servers should return one entry PASSED 
8,48 tno: 256 server_count 1 == 1 for state idle PASSED 
8,56 tno: 257 server_count 1 == 1 for state idle PASSED 
9,68 tno: 258 server s1 with state idle found PASSED 
9,75 tno: 259 server s1 with state idle found PASSED 
11,41 tno: 260 crm_stat(stotal=1, sidle=1, sactive=0, sfail=0, sdeleted=0, vtotal=0, vprovision=0, vactive=0, vidle=0, vdeleted=0) == crm_stat(stotal=1, sidle=1, sactive=0, sfail=0, sdeleted=0, vtotal=0, vprovision=0, vactive=0, vidle=0, vdeleted=0) PASSED 
13,41 tno: 261 crm_stat(stotal=1, sidle=0, sactive=0, sfail=1, sdeleted=0, vtotal=0, vprovision=0, vactive=0, vidle=0, vdeleted=0) == crm_stat(stotal=1, sidle=0, sactive=0, sfail=1, sdeleted=0, vtotal=0, vprovision=0, vactive=0, vidle=0, vdeleted=0) PASSED 
14,68 tno: 262 server s1 with state fail found PASSED 
14,

# CRMVM test


In [1]:
import os
import sys
from mylist import *
from mycrm import *
from myserver import *
from myvm import *
from inspect import currentframe, getframeinfo

# test.py for toy cloud assignment.
# feel free to comment the relevant parts in main function.

## test helper functions

global test_no
global crm
test_no=0
def mycheck(result, msg, lno=0):
    global test_no
    test_no += 1
    cf=currentframe()
    l1=cf.f_back.f_lineno
    if lno:
        l=f'{lno},{l1}'
    else:
        l=f'{l1}'
    if result:
        print(f'{l} tno: {test_no} {msg} PASSED ')
    else:
        print(f'{l}: tno: {test_no} {msg} FAILED ')
        sys.exit(1)

# template crmstat to copy and change values.
#mys=crm_stat(stotal=1,sidle=1,sactive=0,sfail=0,vtotal=1,vprovision=1,vactive=0,'vidle', vdeleted=0)
#print(mys)


def check_stat(s1):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_stat()
    mycheck((s==s1),f'{s1} == {s}', lno)

def check_server_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers(state)
    mycheck(len(s) == count, f'server_count {len(s)} == {count} for state {state}', lno)

    s=crm.get_servers('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'server_count {c} == {count} for state {state}', lno)

def check_server_state(sname, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    assert(state != 'any')
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)
    s=crm.get_servers(state)
    f=0
    for i in s:
        if i.sc.name == sname and i.state == state:
            f=1
            break
    mycheck(f==1, f'server {sname} with state {state} found', lno)

def check_vm_count(count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_vms(state)
    mycheck(len(s) == count, f'vm count {len(s)} == {count} for state {state}', lno)

    s=crm.get_vms('any')
    c=0
    for i in s:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms(state)
        for j in v:
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            if state != 'any' and j.state != state:
                continue
            c += 1
    mycheck(c == count, f'vm count {c} == {count} for state {state}', lno)



def check_server_vm_count(sr, count, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=fs.get_vms('any')
    c=0
    for i in v:
        if state != 'any' and i.state != state:
            continue
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 
    if state == 'any':
        return
    v=fs.get_vms(state)
    c=0
    for i in v:
        c += 1
    mycheck(c==count, f'vm count in {sr} {c} == {count} for state={state}', lno) 

def check_server_vm_state(sr, vm, state):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)
    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    if f != 1:
        mycheck(0, f'vm {vm} in {sr} lookup', lno)
    mycheck(fv.state==state, f'vm: {fv.vc.name} state {fv.state} == {state}', lno) 

def check_novm(vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno

    s=crm.get_vms('any')
    c=0
    for i in s:
        if i.vc.name == vm:
            c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)
    c=0
    s=crm.get_servers('any')
    for i in s:
        v=i.get_vms('any')
        for j in v:
            print(j.vc.name,j.state)
            if j.vc.name == vm:
                c += 1
    mycheck(c == 0, f'vm {vm} not present', lno)

def check_server_novm(sr, vm):
    cf=currentframe()
    lno=cf.f_back.f_lineno
    s=crm.get_servers('any')
    f=0
    for i in s:
        if i.sc.name == sr:
            f=1
            fs=i
            break
    if f != 1:
        mycheck(0, f'server {sr} lookup', lno)

    v=crm.get_vms('any')
    f=0
    for i in v:
        if i.vc.name == vm:
            f=1
            fv=i
            break
    mycheck(f==0, f'vm: {vm}  in server {sr} not present', lno) 

global crm
crm=mycrm()
a = crm.add("s1", 500, 1000)
print(a)
check_server_state("s1", 'idle')
v=crm.provision_vm("vm1", 2, 50)
print(v)
check_server_state("s1", 'idle')
check_stat(crm_stat(stotal=1,sidle=1,vtotal=1,vprovision=1))
check_vm_count(1, 'provision')
check_vm_count(1, 'any')
check_server_vm_state("s1", "vm1", 'provision')
check_server_vm_count("s1", 1, 'provision')
crm.commit_provisioned_vms()
check_server_state("s1", 'active')
check_server_vm_count("s1", 1, 'active')
check_server_vm_state("s1", "vm1", 'active')
check_stat(crm_stat(stotal=1,sactive=1,vtotal=1,vactive=1))
check_vm_count(1, 'any')
check_vm_count(1, 'active')
te=0
try:
    crm.delete_server("s1")
except Exception as e:
    te=1
mycheck(te == 1, 'Cannot delete an active server')

# provision one more vm and then undo it.
v=crm.provision_vm("vm2", 2, 50)
print(v)
check_stat(crm_stat(stotal=1,sactive=1,vtotal=2,vprovision=1,vactive=1))
check_vm_count(2, 'any')
check_vm_count(1, 'active')
check_vm_count(1, 'provision')
check_server_vm_count("s1", 1, 'active')
check_server_vm_state("s1", "vm1", 'active')
check_server_vm_count("s1", 1, 'provision')
check_server_vm_state("s1", "vm2", 'provision')
crm.provision_undo(1)
check_stat(crm_stat(stotal=1,sactive=1,vtotal=1,vactive=1,vdeleted=1))

check_vm_count(1, 'any')
check_server_vm_count("s1", 1, 'any')
check_server_vm_count("s1", 1, 'active')
check_novm("vm2")
check_server_novm("s1", "vm2")
te=0
try:
    crm.delete_vm('random')
except Exception as e:
    te=1
mycheck(te == 1, 'Wrong name to delete')
crm.delete_vm("vm1")
check_vm_count(0, 'any')
check_server_vm_count("s1", 0, 'any')
check_novm("vm1")
check_server_novm("s1", "vm1")
check_stat(crm_stat(stotal=1,sidle=1,vdeleted=2))
crm.delete_server(a.sc.name)
check_stat(crm_stat(vdeleted=2,sdeleted=1))
check_server_count(0, 'any')
check_vm_count(0, 'any')
print(f'********crm_1vm tests done ************')
print(f'********crm_1vm tests done ************')


Server Name: s1 
Server memory : 1000 
Server CPU: 500 
Server State: idle
Virtual Machines: 0
207,68 tno: 1 server s1 with state idle found PASSED 
207,75 tno: 2 server s1 with state idle found PASSED 
VM Name: vm1 
VM memory : 50 
VM CPU: 2 
Server State: provision
Bound Server: 
Server Name: s1 
Server memory : 950 
Server CPU: 498 
Server State: idle
Virtual Machines: 1


210,68 tno: 3 server s1 with state idle found PASSED 
210,75 tno: 4 server s1 with state idle found PASSED 
211,41 tno: 5 crm_stat(stotal=1, sidle=1, sactive=0, sfail=0, sdeleted=0, vtotal=1, vprovision=1, vactive=0, vidle=0, vdeleted=0) == crm_stat(stotal=1, sidle=1, sactive=0, sfail=0, sdeleted=0, vtotal=1, vprovision=1, vactive=0, vidle=0, vdeleted=0) PASSED 
212,81 tno: 6 vm count 1 == 1 for state provision PASSED 
212,89 tno: 7 vm count 1 == 1 for state provision PASSED 
212,96 tno: 8 vm count 1 == 1 for state provision PASSED 
212,105 tno: 9 vm count 1 == 1 for state provision PASSED 
213,81 tno: 10 vm count

SystemExit: 1

/home/blink/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
